In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA

from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, RandomForestRegressor

import xgboost as xgb
from xgboost import XGBClassifier

In [ ]:
pwd

In [ ]:
train = pd.read_csv('./AnnualReturns/train.csv')
test = pd.read_csv('./AnnualReturns/test.csv')

In [ ]:
train_objs_num = len(train)
train = pd.concat(objs=[train, test], axis=0)

In [ ]:
train.shape

In [ ]:
train.head()

In [ ]:
train.info()

In [ ]:
train['return'].describe()

In [ ]:
sns.distplot(train['return']*100, kde=False, bins=100)

In [ ]:
#sns.jointplot(x='sell_date',y='return',data=train)

In [ ]:
train['country_code'].value_counts()

In [ ]:
#sns.jointplot(x='office_id',y='return',data=train)

In [ ]:
train.columns

In [ ]:
#Missing Data
total = train.isnull().sum().sort_values(ascending=False)
percent = (train.isnull().sum()/train.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(10)

In [ ]:
#train['libor_rate'].value_counts()

In [ ]:
#add missing data
train['hedge_value'].fillna(False, inplace=True)
train['indicator_code'].fillna(False, inplace=True)
train['status'].fillna(False, inplace=True)
train['libor_rate'].fillna(train['libor_rate'].mean(), inplace=True)
train['sold'].fillna(0, inplace=True)
train['bought'].fillna(0, inplace=True)

In [ ]:
train['start_date'] = pd.to_datetime(train['start_date'],format='%Y%m%d')
train['creation_date'] = pd.to_datetime(train['creation_date'],format='%Y%m%d')
train['sell_date'] = pd.to_datetime(train['sell_date'],format='%Y%m%d')

In [ ]:
train.head()

In [ ]:
#((train['sell_date'] - train['start_date']) / np.timedelta64(1, 'D')).astype(int)

In [ ]:
#calculate days of investment
train['days_invested'] = ((train['sell_date'] - train['start_date']) / np.timedelta64(1, 'D')).astype(int)

In [ ]:
#calculate amount earned on investment
train['amount_earned'] = (train['sold'] - train['bought'])

In [ ]:
train.columns

In [ ]:
train.drop(['desk_id','start_date','sold','bought','creation_date','sell_date'],axis=1,inplace=True)

In [ ]:
train.head()

In [ ]:
train.info()

In [ ]:
sns.jointplot(x='amount_earned',y='return',data=train)

In [ ]:
train_features = [x for x in train.columns if x not in ['transaction_id','target']]

In [ ]:
#Encode the data
for column in train_features:
    if train[column].dtype == type(object):
        le = LabelEncoder()
        le.fit(train[column])
        train[column] = le.transform(train[column])

In [ ]:
cat_cols = ['office_id', 'pf_category', 'country_code', 'currency', 'type']

In [ ]:
dataset_preprocessed = pd.get_dummies(train, columns = cat_cols)

In [ ]:
train_features = [x for x in dataset_preprocessed.columns if x not in ['portfolio_id','return']]

In [ ]:
dataset_preprocessed.head()

In [ ]:
train_preprocessed = dataset_preprocessed[:train_objs_num]
test_preprocessed = dataset_preprocessed[train_objs_num:]

In [ ]:
X = train_preprocessed[train_features].values
y = train_preprocessed.loc[:,'return'].values
T = test_preprocessed[train_features].values

In [ ]:
y

In [ ]:
from sklearn.preprocessing import StandardScaler
normalizer = StandardScaler()
normalizer.fit_transform(X)

In [ ]:
y

In [ ]:
stacker = RandomForestRegressor(n_estimators=150,random_state=0)
results = cross_val_score(stacker, X, y, cv=5, scoring='r2')
print(results)
print("Stacker score: {} ".format(results.mean()))